In [1]:
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import torchvision.transforms as T
from sklearn.model_selection import KFold

import timm
from tqdm import tqdm

In [2]:
#. 設定
class Config:
  DATA_DIR = "/content/drive/MyDrive/kaggle/csiro_img2bio/data/"
  OUTPUT_DIR = "/content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet"

  IMG_SIZE = 1000
  BATCH_SIZE = 8
  NUM_WORKERS = 4

  MODEL_NAME = "efficientnet_b0"
  OUTPUT_DIM = 5
  N_FOLDS = 5
  LR = 1e-4
  MAX_EPOCHS = 10

  DEBUG = False
  SEED = 42

  USE_TTA_VALID = True #. valid時もTTAを使うか
  USE_LOG1P = True

TARGET_COLS = [
    "Dry_Green_g",
    "Dry_Dead_g",
    "Dry_Clover_g",
    "GDM_g",
    "Dry_Total_g",
]

#. 乱数シードの固定
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

#. configクラスの読み込み
cfg = Config()
seed_everything(cfg.SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [3]:
#. 評価指標
def weighted_r2_score(y_true: np.ndarray, y_pred: np.ndarray):
    """
    y_true, y_pred: shape (N, 5)
    """
    weights = np.array([0.1, 0.1, 0.1, 0.2, 0.5])
    r2_scores = []
    for i in range(5):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]
        ss_res = np.sum((y_t - y_p) ** 2)
        ss_tot = np.sum((y_t - np.mean(y_t)) ** 2)
        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
        r2_scores.append(r2)
    r2_scores = np.array(r2_scores)
    weighted_r2 = np.sum(r2_scores * weights) / np.sum(weights)
    return weighted_r2, r2_scores

In [4]:
#. transform 画像前処理＋Augmentation
def get_train_transform(img_size: int):
    """
    Train用の画像前処理 & Augmentation
    """
    return T.Compose(
        [
            T.Resize((img_size, img_size)),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),
            T.ColorJitter(
                brightness=0.1,
                contrast=0.1,
                saturation=0.1,
                hue=0.05,
            ),
            # ここに強めAugを足していくことも可能
            # T.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            # T.RandomRotation(15),
            T.ToTensor(),
            T.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    )


def get_valid_transform(img_size: int):
    """
    Valid/Test用の画像前処理（Augmentなし）
    """
    return T.Compose(
        [
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    )

In [5]:
#. DataLoader
def create_dataloader(
    df: pd.DataFrame,
    img_root: str,
    transform,
    batch_size: int,
    num_workers: int,
    shuffle: bool,
    drop_last: bool = False,
):
    dataset = ImageRegressionDataset(
        df=df,
        img_root=img_root,
        transform=transform,
    )

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=drop_last,
    )
    return loader


In [6]:
# データセットの作成
class ImageRegressionDataset(Dataset):
    def __init__(self, df, img_root, transform=None):
        self.df = df
        self.img_root = Path(img_root)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_root / row["image_path"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        targets = torch.tensor([
            row[col] for col in TARGET_COLS
        ], dtype=torch.float32)
        return image, targets

In [7]:
#. データの読み込み
def dataload(DATA_DIR, DEBUG):
  train_path = os.path.join(DATA_DIR, "train.csv")
  test_path = os.path.join(DATA_DIR, "test.csv")
  train_df = pd.read_csv(train_path)
  test_df = pd.read_csv(test_path)

  #. DEBUG==TRUEの場合、実験用に学習データを絞る
  if DEBUG:
    train_df = train_df.head(50)

  #. targetを横一列に変換
  train_df = pd.pivot_table(train_df, index='image_path', columns=['target_name'], values='target').reset_index()

  return train_df, test_df

In [8]:
class MultiRegressionModel(nn.Module):
    def __init__(self, model_name: str, output_dim: int, pretrained: bool = True):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=output_dim,
        )

    def forward(self, x):
        return self.backbone(x)  # (B, 5)

In [9]:
def tta_transforms_batch(x: torch.Tensor):
    """
    x: (B, C, H, W)
    return: (T, B, C, H, W)
    """
    xs = [x]
    xs.append(torch.flip(x, dims=[-1]))        # 水平
    xs.append(torch.flip(x, dims=[-2]))        # 垂直
    xs.append(torch.flip(x, dims=[-1, -2]))    # 両方
    return torch.stack(xs, dim=0)

In [10]:

def predict_with_tta_batch(
    model: nn.Module, images: torch.Tensor, device: torch.device
) -> torch.Tensor:
    """
    images: (B, C, H, W), normalize 済み
    return: (B, 5) TTA平均済み
    """
    model.eval()
    images = images.to(device, non_blocking=True)

    tta_imgs = tta_transforms_batch(images)  # (T, B, C, H, W)
    num_tta = tta_imgs.shape[0]

    preds_sum = 0.0
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
        for t in range(num_tta):
            out = model(tta_imgs[t])  # (B, 5)
            preds_sum += out

    preds = preds_sum / num_tta
    return preds

In [11]:
def train_one_fold(
    fold: int,
    train_df: pd.DataFrame,
    valid_df: pd.DataFrame,
    cfg: Config,
    device: torch.device,
) -> float:
    print(f"\n========== Fold {fold} ==========")

    # Transform を関数から取得
    train_transform = get_train_transform(cfg.IMG_SIZE)
    valid_transform = get_valid_transform(cfg.IMG_SIZE)

    # DataLoader も関数から生成
    train_loader = create_dataloader(
        df=train_df,
        img_root=cfg.DATA_DIR,
        transform=train_transform,
        batch_size=cfg.BATCH_SIZE,
        num_workers=cfg.NUM_WORKERS,
        shuffle=True,
        drop_last=False,
    )

    valid_loader = create_dataloader(
        df=valid_df,
        img_root=cfg.DATA_DIR,
        transform=valid_transform,
        batch_size=cfg.BATCH_SIZE,
        num_workers=cfg.NUM_WORKERS,
        shuffle=False,
        drop_last=False,
    )

    # モデルなど
    model = MultiRegressionModel(
        model_name=cfg.MODEL_NAME,
        output_dim=cfg.OUTPUT_DIM,
        pretrained=True,
    ).to(device)

    criterion = nn.SmoothL1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=cfg.MAX_EPOCHS
    )

    scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

    best_score = -1e9
    best_state_dict = None

    # エポックループ
    for epoch in range(1, cfg.MAX_EPOCHS + 1):
        print(f"\nEpoch [{epoch}/{cfg.MAX_EPOCHS}]")

        # ----- Train -----
        model.train()
        train_losses = []

        for images, targets in tqdm(train_loader, desc="Train", leave=False):
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
              if cfg.USE_LOG1P:
                targets = torch.log1p(targets)
                outputs = model(images)
                loss = criterion(outputs, targets)
              else:
                outputs = model(images)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss.item())

        train_loss = float(np.mean(train_losses))
        print(f"Train Loss: {train_loss:.4f}")

        # ----- Valid -----
        model.eval()
        valid_losses = []
        all_preds = []
        all_trues = []

        with torch.no_grad():
            for images, targets in tqdm(
                valid_loader, desc="Valid", leave=False
            ):
                targets = targets.to(device, non_blocking=True)

                if cfg.USE_TTA_VALID:
                    outputs = predict_with_tta_batch(model, images, device)
                else:
                    images = images.to(device, non_blocking=True)
                    with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
                        outputs = model(images)

                        if cfg.USE_LOG1P: #. log1p
                            targets_log = torch.log1p(targets)
                            loss = criterion(outputs, targets_log)
                        else:
                            loss = criterion(outputs, targets)

                valid_losses.append(loss.item())

                if cfg.USE_LOG1P:
                    preds_raw = torch.expm1(outputs)
                    preds_raw = torch.clamp_min(preds_raw, 0.0)
                else:
                    preds_raw = outputs

                all_preds.append(preds_raw.detach().cpu())
                all_trues.append(targets.detach().cpu())

        valid_loss = float(np.mean(valid_losses))
        all_preds_np = torch.cat(all_preds).numpy()
        all_trues_np = torch.cat(all_trues).numpy()
        weighted_r2, r2s = weighted_r2_score(all_trues_np, all_preds_np)

        print(
            f"Valid Loss: {valid_loss:.4f}, "
            f"Weighted R2: {weighted_r2:.4f}, "
            f"R2s: {', '.join([f'{x:.4f}' for x in r2s])}"
        )

        scheduler.step()

        if weighted_r2 > best_score:
            best_score = weighted_r2
            best_state_dict = {
                k: v.cpu().clone() for k, v in model.state_dict().items()
            }
            print(f"  >> Best model updated! (score={best_score:.4f})")

    # 保存
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    save_path = os.path.join(cfg.OUTPUT_DIR, f"model_fold{fold}_best.pth")
    torch.save(best_state_dict, save_path)
    print(f"Best model for fold {fold} saved to: {save_path}")
    print(f"Best Weighted R2 for fold {fold}: {best_score:.4f}")

    return best_score

In [12]:
df_pivot, _ = dataload(cfg.DATA_DIR, DEBUG=False)

print("Pivoted train shape:", df_pivot.shape)
display(df_pivot.head())

kf = KFold(n_splits=cfg.N_FOLDS, shuffle=True, random_state=cfg.SEED)
fold_scores = []

for fold, (tr_idx, va_idx) in enumerate(kf.split(df_pivot)):
    train_df = df_pivot.iloc[tr_idx].reset_index(drop=True)
    valid_df = df_pivot.iloc[va_idx].reset_index(drop=True)

    score = train_one_fold(fold, train_df, valid_df, cfg, device)
    fold_scores.append(score)

print("\n========== CV Result ==========")
for i, s in enumerate(fold_scores):
    print(f"Fold {i}: {s:.4f}")
print(f"Mean Weighted R2: {np.mean(fold_scores):.4f}")

Pivoted train shape: (357, 6)


target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605



========== Fold 0 ==========


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

...

Valid Loss: 0.1276, Weighted R2: -3.2829, R2s: 0.4856, 0.5405, -0.3046, 0.3539, -6.8517

Epoch [9/10]


Train Loss: 0.1682


Valid Loss: 0.1593, Weighted R2: -0.6288, R2s: 0.6530, 0.5219, 0.1459, 0.4054, -1.6839
  >> Best model updated! (score=-0.6288)

Epoch [10/10]


Train Loss: 0.1715


Valid Loss: 0.1384, Weighted R2: -5.7809, R2s: 0.7157, 0.4298, 0.2212, 0.4038, -11.9967
Best model for fold 4 saved to: /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold4_best.pth
Best Weighted R2 for fold 4: -0.6288

========== CV Result ==========
Fold 0: -0.1780
Fold 1: 0.3026
Fold 2: 0.1927
Fold 3: 0.3884
Fold 4: -0.6288
Mean Weighted R2: 0.0154


In [13]:
#. Test用データセット作成
class TestImageDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_root: str, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_root = Path(img_root)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_root / row["image_path"]
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image, row["image_path"]


def create_test_dataloader(
    df: pd.DataFrame,
    img_root: str,
    img_size: int,
    batch_size: int,
    num_workers: int,
):
    """
    Test用 DataLoader を作成
    """
    test_transform = get_valid_transform(img_size)

    dataset = TestImageDataset(
        df=df,
        img_root=img_root,
        transform=test_transform,
    )

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False,
    )
    return loader


In [19]:
#. fold × TTA アンサンブルで test 予測
test_csv = os.path.join(cfg.DATA_DIR, "test.csv")
sub_csv  = os.path.join(cfg.DATA_DIR, "sample_submission.csv")

test_df = pd.read_csv(test_csv)
sub_df  = pd.read_csv(sub_csv)

print("test_df shape:", test_df.shape)
display(test_df.head())
print("sample_submission shape:", sub_df.shape)
display(sub_df.head())

# 画像単位にユニーク化（1 image_path につき1予測を出すため）
test_images = test_df[["image_path"]].drop_duplicates().reset_index(drop=True)
print("unique test images:", len(test_images))

# DataLoader 作成
test_loader = create_test_dataloader(
    df=test_images,
    img_root=cfg.DATA_DIR,
    img_size=cfg.IMG_SIZE,
    batch_size=cfg.BATCH_SIZE,
    num_workers=cfg.NUM_WORKERS,
)

# foldモデル読み込み
models = []
for fold in range(cfg.N_FOLDS):
    model_path = os.path.join(cfg.OUTPUT_DIR, f"model_fold{fold}_best.pth")
    if not os.path.exists(model_path):
        print(f"[Warning] {model_path} not found. skip this fold.")
        continue

    model = MultiRegressionModel(
        model_name=cfg.MODEL_NAME,
        output_dim=cfg.OUTPUT_DIM,
        pretrained=False,  # 推論時なのでpretrainedフラグは不要
    )
    state_dict = torch.load(model_path, map_location="cpu")
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    models.append(model)
    print(f"Loaded model from {model_path}")

if len(models) == 0:
    raise RuntimeError("No model loaded. Check model paths.")


# 画像ごとの予測を計算: image_path -> (5,) ベクトル
image_to_pred = {}

for images, paths in tqdm(test_loader, desc="Test Inference"):
    # fold × TTA アンサンブル
    batch_preds_sum = None

    for model in models:
        preds = predict_with_tta_batch(model, images, device)  # (B, 5)

        preds_np = preds.cpu().numpy()
        if batch_preds_sum is None:
            batch_preds_sum = preds_np
        else:
            batch_preds_sum += preds_np

    # fold平均
    batch_preds_mean = batch_preds_sum / len(models)  # (B, 5)

    # log1p → 元スケール
    preds_raw = np.expm1(batch_preds_mean)
    preds_raw = np.clip(preds_raw, 0.0, None)   # 念のため0未満は0に

    # image_pathごとに保存
    for p, pred in zip(paths, preds_raw):
        image_to_pred[p] = pred  # pred: np.ndarray shape (5,)

test_df shape: (5, 3)


,sample_id,image_path,target_name
0,ID1001187975__Dry_Clover_g,test/ID1001187975.jpg,Dry_Clover_g
1,ID1001187975__Dry_Dead_g,test/ID1001187975.jpg,Dry_Dead_g
2,ID1001187975__Dry_Green_g,test/ID1001187975.jpg,Dry_Green_g
3,ID1001187975__Dry_Total_g,test/ID1001187975.jpg,Dry_Total_g
4,ID1001187975__GDM_g,test/ID1001187975.jpg,GDM_g


sample_submission shape: (5, 2)


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.0
1,ID1001187975__Dry_Dead_g,0.0
2,ID1001187975__Dry_Green_g,0.0
3,ID1001187975__Dry_Total_g,0.0
4,ID1001187975__GDM_g,0.0


unique test images: 1
Loaded model from /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold0_best.pth
Loaded model from /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold1_best.pth
Loaded model from /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold2_best.pth
Loaded model from /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold3_best.pth
Loaded model from /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/model_fold4_best.pth


Test Inference:   0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipython-input-3073180161.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
Test Inference: 100%|██████████| 1/1 [00:00<00:00,  1.85it/s]

{'test/ID1001187975.jpg': array([30.3  , 26.61 ,  0.578, 23.7  , 49.7  ], dtype=float16)}


In [20]:
# submissionの作成

target_name_to_idx = {name: i for i, name in enumerate(TARGET_COLS)}
target_name_to_idx

preds = []

for _, row in test_df.iterrows():
    img_path    = row["image_path"]   # 例: train/ID1001187975.jpg
    target_name = row["target_name"]  # 例: Dry_Clover_g

    if img_path not in image_to_pred:
        raise KeyError(f"{img_path} not found in image_to_pred")

    idx = target_name_to_idx[target_name]      # 0〜4
    pred_value = image_to_pred[img_path][idx]  # そのターゲットの予測値
    preds.append(pred_value)

# 提出用 DataFrame：sample_id + target の2列だけ
submission = test_df[["sample_id"]].copy()

submission["target"] = preds

save_path = os.path.join(cfg.OUTPUT_DIR, "submission.csv")
submission.to_csv(save_path, index=False)

print("Saved submission to:", save_path)
display(submission.head())


Saved submission to: /content/drive/MyDrive/kaggle/csiro_img2bio/result/ex002_251204_efficientnet/submission.csv


/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,sample_id,target
0,ID1001187975__Dry_Clover_g,0.578125
1,ID1001187975__Dry_Dead_g,26.609375
2,ID1001187975__Dry_Green_g,30.296875
3,ID1001187975__Dry_Total_g,49.687500
4,ID1001187975__GDM_g,23.703125


In [21]:
from google.colab import runtime
runtime.unassign()